<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to resize EBS volume using unSkript legos.</b>
</div>

<br>

<center><h2>Resize EBS Volume</h2></center>

# Steps Overview
1) Extract the instanceID and device name from the AWS Cloudwatch alert.
2) Get the PrivateIP address and ssh key pair for the instance.
3) Since we don't support ssh key pair to credential mapping at this time, this runbook will need to be configured with the ssh credential corresponding to the instance(s).
4) Extract the parent block device from the partition name.
5) Extract the EBS volume corresponding to the parent block device.
6) Modify the EBS volume by the SizeToIncreaseBy amount.
7) Extend the file system 

- [ ] This runbook ONLY works for linux instances.
- [ ] The instance needs to have ebsnvme-id installed. This is used to figure out the nvme block device to EBS volumeID.
- [ ] It only supports xfs file system.

In [5]:
# Get the instance ID and device name
for dim in AlertObject['Trigger']['Dimensions']:
    if dim['name'] == 'InstanceId':
        InstanceID = dim['value']
    if dim['name'] == 'device':
        DeviceName = "/dev/"+dim['value']
    if dim['name'] == 'fstype':
        FSType = dim['value']

# Get the region from the AlarmArn
alarmArn = AlertObject['AlarmArn']
Region = alarmArn.split(':')[3]
print(f'InstanceID {InstanceID} Region {Region} FSType {FSType} Device {DeviceName}')

Here we will use unSkript Get AWS Instance Details Lego. This lego takes instance_id and region as input. This input is used to discover all the details of EC2 instance.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_get_instance_details(
    handle,
    instance_id: str,
    region: str,
):

    ec2client = handle.client('ec2', region_name=region)
    instances = []
    response = ec2client.describe_instances(
        Filters=[{"Name": "instance-id", "Values": [instance_id]}])
    for reservation in response["Reservations"]:
        for instance in reservation["Instances"]:
            instances.append(instance)

    return instances[0]


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "instance_id": "InstanceID",
    "region": "Region"
    }''')
task.configure(outputName="InstanceDetail")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_get_instance_details, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [7]:
# lsblk -no pkname /dev/nvme0n1p1
PrivateIPAddress = InstanceDetail.get('PrivateIPAddress')

Here we will use unSkript SSH Execute Remote Command Lego. This lego takes hosts, command and sudo as input. This input is used to SSH Execute Remote Command and get the block device details.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import json
import tempfile
import os
from pydantic import BaseModel, Field
from pssh.clients import ParallelSSHClient
from typing import List, Optional
from unskript.connectors import ssh

from unskript.legos.cellparams import CellParams
from unskript import connectors


from beartype import beartype
@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False):

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)
        res[hostname] = output

        o = "\n".join(output)
        print(f"Output from host {hostname}\n{o}\n")

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "\\"lsblk -no pkname \\" + DeviceName",
    "hosts": "[PrivateIPAddress]",
    "sudo": "True"
    }''')
task.configure(outputName="BlockDeviceDetail")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(ssh_execute_remote_command, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [9]:
BlockDeviceName = BlockDeviceDetail[PrivateIPAddress][0]
print(BlockDeviceName)

Here we will use unSkript SSH Execute Remote Command Lego. This lego takes hosts, command and sudo as input. This input is used to SSH Execute Remote Command and get the EBS volume details.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import json
import tempfile
import os
from pydantic import BaseModel, Field
from pssh.clients import ParallelSSHClient
from typing import List, Optional
from unskript.connectors import ssh

from unskript.legos.cellparams import CellParams
from unskript import connectors


from beartype import beartype
@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False):

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)
        res[hostname] = output

        o = "\n".join(output)
        print(f"Output from host {hostname}\n{o}\n")

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "\\"/sbin/ebsnvme-id \\" + DeviceName",
    "hosts": "[PrivateIPAddress]",
    "sudo": "True"
    }''')
task.configure(outputName="EBSVolumeDetail")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(ssh_execute_remote_command, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [11]:
EBSVolume = EBSVolumeDetail[PrivateIPAddress][0].split(":")[1].lstrip()
print(EBSVolume)

Here we will use unSkript EBS Modify Volume Lego. This lego takes volume_id, resize_option, resize_value and region as input. This input is used to Modify/Resize volume for Elastic Block Storage (EBS).

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import enum
from pydantic import BaseModel, Field
from unskript.legos.aws.aws_get_handle import Session
from polling2 import poll_decorator


class SizingOption(enum.Enum):
    Add = "Add"
    Mutiple = "Multiple"


from beartype import beartype
@beartype
def aws_ebs_modify_volume(
    hdl: Session,
    volume_id: str,
    resize_option: str,
    resize_value: float,
    region: str,
):
    """aws_ebs_modify_volume modifies the size of the EBS Volume.
    You can either increase it a provided value or by a provided multiple value.

    :type volume_id: string
    :param volume_id: ebs volume id.

    :type resize_option: string
    :param resize_option: option to resize the volume, by a fixed amount or by a multiple of the existing size.

    :type value: int
    :param value: The value by which the volume should be modified, depending upon the resize option.

    :type region: string
    :param region: AWS Region of the volume.

    :rtype: New volume size.
    """

    ec2Client = hdl.client("ec2", region_name=region)
    ec2Resource = hdl.resource("ec2", region_name=region)
    # Get the current volume size.
    Volume = ec2Resource.Volume(volume_id)
    currentSize = Volume.size

    if resize_option == SizingOption.Add.value:
        newSize = currentSize + resize_value
    elif resize_option == SizingOption.Mutiple.value:
        newSize = currentSize * resize_value

    print(f'CurrentSize {currentSize}, NewSize {newSize}')

    resp = ec2Client.modify_volume(
        VolumeId=volume_id,
        Size=newSize)

    # Check the modification state
    try:
        check_modification_status(ec2Client, volume_id)
    except Exception as e:
        print(f'Modify volumeID {volume_id} failed: {str(e)}')
        raise e

    print(f'Volume {volume_id} size modified successfully to {newSize}')
    return


@poll_decorator(step=60, timeout=600, check_success=lambda x: x is True)
def check_modification_status(ec2Client, volumeID) -> bool:
    resp = ec2Client.describe_volumes_modifications(VolumeIds=[volumeID])
    state = resp['VolumesModifications'][0]['ModificationState']
    progress = resp['VolumesModifications'][0]['Progress']
    print(f'Volume modification state {state}, Progress {progress}')
    if state == 'completed' or state == None:
        return True
    elif state == 'failed':
        raise Exception("Get Status Failed")
    return False


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region",
    "resize_option": "\\"Add\\"",
    "volume_id": "EBSVolume"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(aws_ebs_modify_volume, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [12]:
# growpart /dev/nvme0n1 1
growpartCommand = "growpart /dev/" + BlockDeviceName + " 1"

Here we will use unSkript SSH Execute Remote Command Lego. This lego takes hosts, command and sudo as input. This input is used to SSH Execute Remote Command.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import json
import tempfile
import os
from pydantic import BaseModel, Field
from pssh.clients import ParallelSSHClient
from typing import List, Optional
from unskript.connectors import ssh

from unskript.legos.cellparams import CellParams
from unskript import connectors


from beartype import beartype
@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False):

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)
        res[hostname] = output

        o = "\n".join(output)
        print(f"Output from host {hostname}\n{o}\n")

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "growpartCommand",
    "hosts": "[PrivateIPAddress]",
    "sudo": "True"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(ssh_execute_remote_command, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

Here we will use unSkript SSH Execute Remote Command Lego. This lego takes hosts, command and sudo as input. This input is used to SSH Execute Remote Command and get Mount path details.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import json
import tempfile
import os
from pydantic import BaseModel, Field
from pssh.clients import ParallelSSHClient
from typing import List, Optional
from unskript.connectors import ssh

from unskript.legos.cellparams import CellParams
from unskript import connectors


from beartype import beartype
@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False):

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)
        res[hostname] = output

        o = "\n".join(output)
        print(f"Output from host {hostname}\n{o}\n")

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "\\"findmnt -nr -o target -S \\" + DeviceName",
    "hosts": "[PrivateIPAddress]",
    "sudo": "False"
    }''')
task.configure(outputName="MountPathDetail")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(ssh_execute_remote_command, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

In [15]:
MountPath = MountPathDetail[PrivateIPAddress][0]

Here we will use unSkript SSH Execute Remote Command Lego. This lego takes hosts, command and sudo as input. This input is used to SSH Execute Remote Command for Mount path.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
import json
import tempfile
import os
from pydantic import BaseModel, Field
from pssh.clients import ParallelSSHClient
from typing import List, Optional
from unskript.connectors import ssh

from unskript.legos.cellparams import CellParams
from unskript import connectors


from beartype import beartype
@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False):

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)
        res[hostname] = output

        o = "\n".join(output)
        print(f"Output from host {hostname}\n{o}\n")

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "\\"xfs_growfs -d \\" + MountPath",
    "hosts": "[PrivateIPAddress]",
    "sudo": "True"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(ssh_execute_remote_command, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

### Conclusion
In this Runbook, we demonstrated the use of unSkript's AWS and SSH legos to Resize EBS Volume and run resizes the EBS volume to a specified amount. This runbook can be attached to Disk usage related Cloudwatch alarms to do the appropriate resizing. It also extends the filesystem to use the new volume size. To view the full platform capabilities of unSkript please visit https://us.app.unskript.io